In [1]:
import requests
import pandas as pd
import folium as f

In [2]:
root = 'https://saopaulo.publicbikesystem.net/ube/gbfs/v1/'
endpoints = requests.get(root).json()['data']['en']['feeds']

In [3]:
main = endpoints[0]
main

{'name': 'station_information',
 'url': 'https://saopaulo-br.publicbikesystem.net/customer/ube/gbfs/v1/en/station_information'}

In [4]:
raw_station_information = requests.get(main['url']).json()['data']['stations']
station_information = pd.DataFrame(raw_station_information)
station_information.head(3)

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,rental_methods,groups,obcn,nearby_distance,post_code,planned_date
0,1,1 - Largo da Batata,REGULAR,-23.566831,-46.693741,0.0,"Av. Brigadeiro Faria Lima, esquina R. Teodoro ...",83,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",[Pinheiros],1,1000.0,NaN,NaN
1,3,3 - CPTM Pinheiros,VAULT,-23.566478,-46.701258,0.0,"R. Gilberto Sabino, 138/ ao lado do ponto de ô...",15,"[KEY, TRANSITCARD, PHONE]",[Pinheiros],3,1000.0,03164200,NaN
2,4,4 - Rua Diogo Moreira,REGULAR,-23.569145,-46.692003,0.0,"Av. Brigadeiro Faria Lima, na altura do número...",23,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",[Pinheiros],4,1000.0,03164200,NaN


In [5]:
huge_stations = station_information.loc[station_information.capacity >= 30]
len(huge_stations), len(station_information)

(6, 259)

In [6]:
def newMap():
    return f.Map(location=[-23.549279718284097, -46.63297420696871], zoom_start=12, tiles="cartodbpositron")

In [7]:
points = huge_stations[['lat', 'lon']].to_records(index=False)
points

rec.array([(-23.56683096, -46.6937412 ), (-23.56616415, -46.69395917),
           (-23.59391757, -46.69126812), (-23.5597721 , -46.7227278 ),
           (-23.59295294, -46.68093275), (-23.592837  , -46.6804    )],
          dtype=[('lat', '<f8'), ('lon', '<f8')])

In [8]:
m = newMap()
for p in points:
    f.CircleMarker(p).add_to(m)
m

In [9]:
m = newMap()
for i, row in station_information.iterrows():
    point = [row.lat, row.lon]
    color='#ffee00'
    if row.capacity >= 30:
        color='#ed1717'
    elif row.capacity >= 20:
        color='#f24d11'
    elif row.capacity >= 15:
        color='#f6830c'
    elif row.capacity >= 10:
        color='#fbb806'
    f.CircleMarker(point, popup=row.address, color=color).add_to(m)
m

In [10]:
m = newMap()
for i, row in station_information.iterrows():
    point = [row.lat, row.lon]
    color='#ffee00'
    if row.altitude > 0:
        color='#ed1717'
    f.CircleMarker(point, popup=str(row.altitude), color=color).add_to(m)
m

In [11]:
groups = [] 
for element in station_information.iterrows():
  if len(element[1].groups) >= 1:
    groups.append(element[1].groups[0])

groups_dict = {x: groups.count(x) for x in groups}

groups_descending_keys = sorted(groups_dict, key=groups_dict.get, reverse=True)

for key in groups_descending_keys:
  print("{:<20} {:<5}".format(key, groups_dict[key]))

Itaim Bibi           38   
Pinheiros            33   
Moema                26   
Brooklin             23   
Jardim Paulista      21   
Vila Mariana         20   
USP                  20   
Bela Vista           12   
Santa Cecilia        11   
Alto de Pinheiros    11   
Barra Funda          8    
Perdizes             7    
Consolação           6    
República            6    
Saúde                4    
Santana              4    
Liberdade            2    
Paraíso              2    
Casa Verde           2    
Sé                   1    
Campo Belo           1    


In [12]:
rental_methods = []
for element in station_information.iterrows():
  for a in range(len(element[1].rental_methods)):
    rental_methods.append(element[1].rental_methods[a])

rental_methods_dict = {x: rental_methods.count(x) for x in rental_methods}

rental_methods_descending_keys = sorted(rental_methods_dict, key=rental_methods_dict.get, reverse=True)

for key in rental_methods_descending_keys:
  print("{:<15} {:<5}".format(key, rental_methods_dict[key]))


KEY             259  
TRANSITCARD     259  
PHONE           259  
CREDITCARD      113  


In [13]:
district = []
for x in station_information.iterrows():
    if 'CREDITCARD' in x[1].rental_methods:
      district.append(x[1].groups[0])

district_dict = {x: district.count(x) for x in district}

district_descending_keys = sorted(district_dict, key=district_dict.get, reverse=True)

for key in district_descending_keys:
  print("{:<20} {:<5}".format(key, district_dict[key]))


Moema                18   
Vila Mariana         15   
Itaim Bibi           14   
Pinheiros            10   
USP                  9    
Santa Cecilia        7    
Alto de Pinheiros    6    
Consolação           5    
República            5    
Jardim Paulista      4    
Bela Vista           4    
Saúde                4    
Brooklin             3    
Barra Funda          3    
Liberdade            1    
Sé                   1    
Campo Belo           1    
Paraíso              1    
Perdizes             1    
Santana              1    
